In [1]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
import os
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models.inception import inception_v3


2024-06-01 15:14:48.278262: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-01 15:14:48.938507: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-01 15:14:48.939621: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 15:14:50.667052: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/robivad/.pyenv/versions/3.8.1/envs/cv381/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Load InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, pooling='avg')


In [3]:
# Load InceptionV3 model
model = inception_v3(pretrained=True, transform_input=False)
model.fc = nn.Identity()
model.eval()

/home/robivad/.pyenv/versions/3.8.1/envs/cv381/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/robivad/.pyenv/versions/3.8.1/envs/cv381/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [4]:
import os
import numpy as np
from scipy.linalg import sqrtm
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from PIL import Image

def load_and_preprocess_image(file_path):
    img = Image.open(file_path).convert('RGB')
    img = img.resize((299, 299))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def calculate_fid(real_dir, generated_dir):
    # Load InceptionV3 model
    # sto usando l'ultimo average pooling layer
    model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
    model = Model(inputs=model.input, outputs=model.layers[-1].output)

    # Load and preprocess images
    real_images = np.concatenate([load_and_preprocess_image(os.path.join(real_dir, f)) for f in os.listdir(real_dir)], axis=0)
    generated_images = np.concatenate([load_and_preprocess_image(os.path.join(generated_dir, f)) for f in os.listdir(generated_dir)], axis=0)

    # Extract features
    real_features = model.predict(real_images)
    generated_features = model.predict(generated_images)

    # Calculate mean and covariance
    mu_real, sigma_real = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu_generated, sigma_generated = generated_features.mean(axis=0), np.cov(generated_features, rowvar=False)

    # Calculate FID
    diff = mu_real - mu_generated
    covmean, _ = sqrtm(sigma_real.dot(sigma_generated), disp=False)

    if np.iscomplexobj(covmean):
        covmean = covmean.real

    fid = diff.dot(diff) + np.trace(sigma_real + sigma_generated - 2 * covmean)
    return fid

# Example usage
real_images_folder = 'world'
generated_images_folders = ['colorized_sample/large', 'colorized_sample/normal', 'colorized_sample/tiny']

for folder in generated_images_folders:
    fid_score = calculate_fid(real_images_folder, folder)
    print(f'FID Score for {folder}: {fid_score}')

# piu' basso e' meglio e'

2024-06-01 15:15:09.383982: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 138392748 exceeds 10% of free system memory.
2024-06-01 15:15:11.268917: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 90935296 exceeds 10% of free system memory.
2024-06-01 15:15:11.353637: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 88510464 exceeds 10% of free system memory.
2024-06-01 15:15:11.458908: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177020928 exceeds 10% of free system memory.
2024-06-01 15:15:11.611672: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 43655168 exceeds 10% of free system memory.


4/4 [==============================] - 7s 2s/step
FID Score for colorized_sample/large: 144.83402127038866
4/4 [==============================] - 7s 2s/step
FID Score for colorized_sample/normal: 114.44366103944091
4/4 [==============================] - 9s 2s/step
FID Score for colorized_sample/tiny: 117.98431280842647
